In [1]:
import pandas as pd
import numpy as np

In [2]:
# comment out unless google colab
from google.colab import files
uploaded = files.upload()

Saving Will_1k_set_1.csv to Will_1k_set_1.csv


In [3]:
# comment out unless google colab
import io
df = pd.read_csv(io.BytesIO(uploaded['Will_1k_set_1.csv']))

In [4]:
df['url'] = df['file_name'].apply(lambda x: 'https://commons.wikimedia.org/wiki/'+x.replace(" ","_"))

In [5]:
df1 = df[['url', 'depicts', 'parsed caption']]

In [6]:
import ast

df1['depicts'] = df1['depicts'].apply(ast.literal_eval)

<ipython-input-6-fd6255286db4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['depicts'] = df1['depicts'].apply(ast.literal_eval)


In [7]:
df1['count_depicts'] = df['depicts'].apply(lambda x: len(ast.literal_eval(x)))

<ipython-input-7-e82ea7bd64ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['count_depicts'] = df['depicts'].apply(lambda x: len(ast.literal_eval(x)))


In [8]:
df1['count_depicts'].value_counts()

count_depicts
1     624
2     166
3      86
4      64
5      35
6      11
8       6
7       5
10      2
9       1
Name: count, dtype: int64

In [9]:
df1['suggestions'] = None

<ipython-input-9-d99b59a87583>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'] = None


In [10]:
# https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c
# https://liaad.github.io/yake/docs/getting_started.html

In [10]:
!pip install yake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00


In [11]:
import yake

In [12]:
for i in range(len(df1)):
  kw_extractor = yake.KeywordExtractor()
  text = str(df1["parsed caption"][i])
  language = "en"
  max_ngram_size = 4
  deduplication_threshold = 0.9
  numOfKeywords = 10
  custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
  keywords = custom_kw_extractor.extract_keywords(text)

  suggestions=[]
  for kw in keywords:
    suggestions.append(kw)
  df1['suggestions'][i] = suggestions

<ipython-input-12-92b289fa510f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-12-92b289fa510f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-12-92b289fa510f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestions'][i] = suggestions
<ipython-input-12-92b289fa510f>:14: SettingWithCopyWarning: 
A value is trying to be 

In [13]:
df1['suggestions'][0]

[('Plac Wielkopolski w Poznaniu', 0.0020632546775457305),
 ('Wielkopolski w Poznaniu', 0.012602360123953448),
 ('Plac Wielkopolski', 0.023458380875189744),
 ('Poznaniu', 0.08596317751626563),
 ('Wielkopolski', 0.1447773057422032),
 ('Plac', 0.15831692877998726)]

In [14]:
df1['suggestion_tags'] = df1['suggestions'].apply(lambda x: [item[0] for item in x])

<ipython-input-14-2778d7505691>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['suggestion_tags'] = df1['suggestions'].apply(lambda x: [item[0] for item in x])


In [15]:
import requests
def get_qnums(tag_list):
  qnum_dict = {}
  for s in tag_list:
    try:
      url = f'https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles={s}&languages=en&props=labels&format=json'
      response = requests.get(url).json()
      qnum = next(iter(response['entities']))
      if qnum.startswith('Q'):
        qnum_dict[s] = qnum
      else:
        #qnum_dict[s] = 'None'
        pass
    except:
      qnum_dict[s] = None
  return qnum_dict

In [16]:
df1['q_tags'] = df1['suggestion_tags'].apply(get_qnums)

<ipython-input-16-87cd6dbe8057>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['q_tags'] = df1['suggestion_tags'].apply(get_qnums)


In [17]:
df1['q_tags']

0       {'Wielkopolski': 'Q1460188', 'Plac': 'Q7200197'}
1                          {'Horizontal bar': 'Q623270'}
2      {'Siege of Lille': 'Q403479', 'France': 'Q142'...
3            {'Waycross Historic District': 'Q30622481'}
4                                                     {}
                             ...                        
995                                                   {}
996    {'Náchod District': 'Q838308', 'Bild': 'Q156203'}
997                                                   {}
998                                  {'Bild': 'Q156203'}
999    {'Edit': 'Q410090', 'Gravel pit': 'Q504874', '...
Name: q_tags, Length: 1000, dtype: object

In [18]:
df1['num_q'] = df1['q_tags'].apply(lambda x: len(x))

<ipython-input-18-2d6bf9d89539>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num_q'] = df1['q_tags'].apply(lambda x: len(x))


In [19]:
df1['num_q'].value_counts()

num_q
0    201
1    172
2    162
4    137
3    121
5    120
6     59
7     23
8      4
9      1
Name: count, dtype: int64

In [30]:
df1['suggestions_q'] = df1.apply(lambda row: [(key, value, row['q_tags'].get(key)) for key, value in row['suggestions'] if row['q_tags'].get(key) is not None], axis=1)

In [35]:
def bests(suggestions_q):
  return [(term, score, q) for term, score, q in suggestions_q if score < 0.05]

In [36]:
df1['best_suggest'] = df1['suggestions_q'].apply(bests)

In [ ]:
df1

In [37]:
df1['url'][40]

'https://commons.wikimedia.org/wiki/File:Gliricidia_sepium_(Fabaceae)_01.jpg'

In [38]:
df1['suggestions'][40]

[('Gliricidia sepium flowers', 0.008321268142839145),
 ('Gliricidia sepium', 0.04940384002065631),
 ('sepium flowers', 0.04940384002065631),
 ('Gliricidia', 0.15831692877998726),
 ('flowers', 0.15831692877998726),
 ('sepium', 0.29736558256021506)]

In [39]:
df1['q_tags'][40]

{'Gliricidia sepium': 'Q2717303', 'Gliricidia': 'Q12549487'}

In [40]:
df1['suggestions_q'][40]

[('Gliricidia sepium', 0.04940384002065631, 'Q2717303'),
 ('Gliricidia', 0.15831692877998726, 'Q12549487')]

In [41]:
df1['best_suggest'][40]

[('Gliricidia sepium', 0.04940384002065631, 'Q2717303')]

In [43]:
df1['depicts'][40]

['Gliricidia sepium', 'flower']

In [45]:
df_result = df1[['url', 'suggestions_q', 'best_suggest']]

In [46]:
df_result

,url,suggestions_q,best_suggest
0,https://commons.wikimedia.org/wiki/File:Plac_W...,"[(Wielkopolski, 0.1447773057422032, Q1460188),...",[]
1,https://commons.wikimedia.org/wiki/File:2018-1...,"[(Horizontal bar, 0.048811108786045235, Q623270)]","[(Horizontal bar, 0.048811108786045235, Q623270)]"
2,https://commons.wikimedia.org/wiki/File:CP_001...,"[(Siege of Lille, 0.02727038941117262, Q403479...","[(Siege of Lille, 0.02727038941117262, Q403479..."
3,https://commons.wikimedia.org/wiki/File:Waycro...,"[(Waycross Historic District, 0.00094817452905...","[(Waycross Historic District, 0.00094817452905..."
4,https://commons.wikimedia.org/wiki/File:J31_86...,[],[]
...,...,...,...
995,https://commons.wikimedia.org/wiki/File:Sammy_...,[],[]
996,https://commons.wikimedia.org/wiki/File:Houses...,"[(Náchod District, 0.003678542013331895, Q8383...","[(Náchod District, 0.003678542013331895, Q8383..."
997,https://commons.wikimedia.org/wiki/File:SDO_20...,[],[]
998,https://commons.wikimedia.org/wiki/File:Site_o...,"[(Bild, 0.012347542149559911, Q156203)]","[(Bild, 0.012347542149559911, Q156203)]"


In [50]:
df_result.to_csv('suggestions_1k.csv', index=False)

In [49]:
from google.colab import files

# Download the CSV file
files.download('suggestions_1k.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>